In [5]:
#dependencies
import pandas as pd
import requests
import json
from pprint import pprint
from config import omdb_api

In [3]:
#import csv, pulled from kaggle (https://www.kaggle.com/abhimanyudasarwar/netflix-originals)
info_file = "resources/netflix_orig_info.csv"
info_df = pd.read_csv (info_file)

info_df.head()

,Title,Genre,Original Network,Premiere,Seasons,Length,Netflix Exclusive Regions,Status
0,House of Cards,Political drama,Netflix,"February 1, 2013","6 seasons, 73 episodes",42–59 min.,Worldwide,Ended
1,Hemlock Grove,Horror,Netflix,"April 19, 2013","3 seasons, 33 episodes",45–58 min.,Worldwide,Ended
2,Orange Is the New Black,Comedy-drama,Netflix,"July 11, 2013","7 seasons, 91 episodes",50–92 min.,Worldwide,Ended
3,Marco Polo,Historical drama,Netflix,"December 12, 2014","2 seasons, 20 episodes",48–65 min.,Worldwide,Ended
4,Bloodline,Thriller,Netflix,"March 20, 2015","3 seasons, 33 episodes",48–68 min.,Worldwide,Ended


In [6]:
#isolate relevant columns
netflix_df = info_df.loc [:, ['Title', 'Genre', 'Seasons']]

#drop titles that are not TV shows
netflix_df = netflix_df.dropna (how = 'any')
netflix_df

,Title,Genre,Seasons
0,House of Cards,Political drama,"6 seasons, 73 episodes"
1,Hemlock Grove,Horror,"3 seasons, 33 episodes"
2,Orange Is the New Black,Comedy-drama,"7 seasons, 91 episodes"
3,Marco Polo,Historical drama,"2 seasons, 20 episodes"
4,Bloodline,Thriller,"3 seasons, 33 episodes"
...,...,...,...
492,Rabbids Invasion,Animation,"1 season, 26 episodes"
493,Money Heist,Thriller,"2 parts, 16 episodes"
494,Top Boy,Crime drama\n,"1 series, 10 episodes"
495,You,Thriller,"1 season, 10 episodes"


In [7]:
#OMDB API call setup
omdb_url = "http://www.omdbapi.com/?type=series&t="
api_key = "&apikey=" + omdb_api

#create new columns for items of interest
netflix_df['Year'] = ''
netflix_df['IMDB_rating'] = ''
netflix_df['Writer(s)'] = ''

#loop through dataframe & perform get requests
for index, row in netflix_df.iterrows():
    response = requests.get (f'{omdb_url}{row["Title"]}{api_key}').json()
    
    #try/except to populate corresponding values
    try:
        netflix_df.loc[index, 'Year'] = response['Year']
        
    except:
        pass
    try:
        netflix_df.loc[index, 'Writer(s)'] = response['Writer']
        
    except:
        pass
    
    try:
        netflix_df.loc[index, 'IMDB_rating'] = response['imdbRating']
        
    except:
        pass

netflix_df

,Title,Genre,Seasons,Year,IMDB_rating,Writer(s)
0,House of Cards,Political drama,"6 seasons, 73 episodes",2013–2018,8.7,Beau Willimon
1,Hemlock Grove,Horror,"3 seasons, 33 episodes",2013–2015,7.1,"Brian McGreevy, Lee Shipman"
2,Orange Is the New Black,Comedy-drama,"7 seasons, 91 episodes",2013–2019,8.1,Jenji Kohan
3,Marco Polo,Historical drama,"2 seasons, 20 episodes",2014–2016,8.0,John Fusco
4,Bloodline,Thriller,"3 seasons, 33 episodes",2015–2017,8.0,"Glenn Kessler, Todd A. Kessler, Daniel Zelman"
...,...,...,...,...,...,...
492,Rabbids Invasion,Animation,"1 season, 26 episodes",2013–2019,5.0,N/A
493,Money Heist,Thriller,"2 parts, 16 episodes",2017–,8.4,Álex Pina
494,Top Boy,Crime drama\n,"1 series, 10 episodes",2011–,8.4,Ronan Bennett
495,You,Thriller,"1 season, 10 episodes",2018–,7.8,"Sera Gamble, Greg Berlanti"
